In [17]:
import pandas as pd
import numpy as np
from faker import Faker
import random
import unicodedata
import random
from gender_guesser.detector import Detector
from datetime import date
fake = Faker('es_ES')
Faker.seed(1000)

In [18]:
df = pd.DataFrame()

In [19]:
df['cliente_id'] = range(1, 100001)

In [20]:
df['nombre_cliente'] = [unicodedata.normalize('NFKD', fake.first_name()).encode('ascii', 'ignore').decode('utf-8') for _ in range(len(df))]

In [21]:
df['apellido_cliente'] = [unicodedata.normalize('NFKD', fake.last_name()).encode('ascii', 'ignore').decode('utf-8') for _ in range(len(df))]

In [22]:
df['dni_cliente'] = [str(fake.nif()) for _ in range(len(df))]

In [23]:
df['email_cliente'] = [f"{nombre.lower()}.{apellido.lower()}@redhorn.com" for nombre, apellido in zip(df['nombre_cliente'], df['apellido_cliente'])]

In [24]:
beta_values = np.random.beta(2, 5, len(df))
edad_minima = 15
edad_maxima = 90
ages = np.round(beta_values * (edad_maxima- edad_minima) + edad_minima)
df['edad_cliente'] = ages.astype(int)

In [25]:
df['numero_telefono'] = [fake.phone_number() for _ in range(len(df))]


In [26]:
df['direccion_cliente'] = [unicodedata.normalize('NFKD', str(fake.address()).encode('ascii', 'ignore').decode('utf-8')) for _ in range(len(df))]


In [27]:
df['trabajo_cliente'] = [str(fake.job()) for _ in range(len(df))]

In [28]:
guesser = Detector()
df['genero_cliente'] = [guesser.get_gender(nombre) for nombre in df['nombre_cliente']]


In [29]:
df['genero_cliente'] = df['genero_cliente'].replace({
    'male': 'hombre',
    'female': 'mujer',
    'unknown': 'desconocido',
    'andy': 'desconocido',
    'mostly_male': 'hombre',
    'mostly_female': 'mujer'
})

In [30]:
df['fecha_registro'] = [fake.date_between(start_date=date(2023, 1, 1), end_date='today') for _ in range(len(df))]


In [31]:
df.head()

,cliente_id,nombre_cliente,apellido_cliente,dni_cliente,email_cliente,edad_cliente,numero_telefono,direccion_cliente,trabajo_cliente,genero_cliente,fecha_registro
0,1,Mayte,Hervia,16378221J,mayte.hervia@redhorn.com,39,+34 972 150 145,"Calle Ruth Pea 40 Piso 3 \nLa Rioja, 21626",Guardián de prisión,mujer,2023-02-14
1,2,Timoteo,Cisneros,70315544J,timoteo.cisneros@redhorn.com,24,+34 976442433,"Pasaje de Valentina Benet 77 Piso 9 \nLen, 45366",Mampostero,hombre,2023-12-11
2,3,Haydee,Alvarez,63010221L,haydee.alvarez@redhorn.com,36,+34842916745,"Avenida de Marino Gibert 270 Piso 1 \nJan, 03720",Declarante de aduana,mujer,2023-11-05
3,4,Martirio,Iniesta,11996524T,martirio.iniesta@redhorn.com,35,+34 921 78 05 60,"Camino de Domingo Merino 34\nPontevedra, 15421",Guardafrenos,hombre,2023-08-10
4,5,Conrado,Belmonte,86277092K,conrado.belmonte@redhorn.com,28,+34 987655388,"Via de igo Dalmau 36\nMlaga, 08333",Meteorólogo,hombre,2023-09-26


In [37]:
from google.cloud import bigquery
from pandas_gbq import to_gbq
import pandas as pd
df['fecha_registro'] = pd.to_datetime(df['fecha_registro'])

schema = [
    {'name': 'cliente_id', 'type': 'INTEGER'},
    {'name': 'nombre_cliente', 'type': 'STRING'},
    {'name': 'apellido_cliente', 'type': 'STRING'},
    {'name': 'dni_cliente', 'type': 'STRING'},
    {'name': 'email_cliente', 'type': 'STRING'},
    {'name': 'edad_cliente', 'type': 'INTEGER'},
    {'name': 'numero_telefono', 'type': 'STRING'},
    {'name': 'direccion_cliente', 'type': 'STRING'},
    {'name': 'trabajo_cliente', 'type': 'STRING'},
    {'name': 'genero_cliente', 'type': 'STRING'},
    {'name': 'fecha_registro', 'type': 'DATETIME'},
]

table_id = 'dataproject-2-413010.BBDD.tabla_clientes'

client = bigquery.Client()

to_gbq(df, destination_table=table_id, project_id='dataproject-2-413010', if_exists='replace', table_schema=schema)
